In [34]:
!pwd
!ls

/d/GH/GitWorkSpace/bank_model_competiton/data/v11
model_stacking_v11.ipynb
process_v11.ipynb
test.dat.v11
train.dat.v11


In [35]:
import pandas as pd 
import matplotlib.pyplot as plt
import statistics
import datetime
import seaborn as sns
import os
import numpy as np
import time 
from sklearn.preprocessing import OrdinalEncoder


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

suffix = os.path.split(os.getcwd())[-1]

root_dir='../../'

train_path=root_dir+'train.csv'
train_tx_path=root_dir+'train_bank_statement.csv'
test_path=root_dir+'testaa.csv'
test_tx_path=root_dir+'testaa_bank_statement.csv'

print(suffix)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

v11
process time :  2025-08-28 14:49:13


In [36]:
mode="train"

NROWS = 100000000000000000

if mode == "train":
    base_input_path = train_path
    tx_input_path = train_tx_path
    output_path = "train.dat.%s" % suffix
else:
    base_input_path = test_path
    tx_input_path = test_tx_path
    output_path = "test.dat.%s" % suffix

print(mode, base_input_path, tx_input_path, output_path)

train ../../train.csv ../../train_bank_statement.csv train.dat.v11


#  基础特征

In [37]:
!pwd
df = pd.read_csv(base_input_path, index_col=['id'], nrows=NROWS)
print(df.head())
print(df.shape)

df['balance_account_avg'] = df['balance'] / df['balance_accounts']
df['loan_term_avg'] = df['loan'] / df['term']
df['balance_accounts_ratio'] = df['balance_accounts'] / df['total_accounts']

df['loan_log'] = df['loan'].apply(lambda x : np.log(x))

#log 处理
df['balance_account_avg_log'] = df['balance_account_avg'].apply(lambda x : np.log(x))
df['loan_term_avg_log'] = df['loan_term_avg'].apply(lambda x : np.log(x))
df['balance_accounts_ratio_log'] = df['balance_accounts_ratio'].apply(lambda x : np.log(x))
df['interest_rate_log'] = df['interest_rate'].apply(lambda x : np.log(x))

df['balance_log'] = df['balance'].apply(lambda x : np.log(x))
df['balance_limit_log'] = df['balance_limit'].apply(lambda x : np.log(x))
df['balance_accounts_log'] = df['balance_accounts'].apply(lambda x : np.log(x))

# interest_rate_labels = ['%d' % i for i in range(20)]
# print(interest_rate_labels)
# df['interest_rate_cut'] = pd.cut(df['interest_rate'], bins=20, labels = interest_rate_labels)

#level 特征
df['level_hash'] = df['level'].apply(lambda x : hash(x) % 1000 )

encoder = OrdinalEncoder(categories=[['A0','A1','A2','A3','A4','A5','B0','B1','B2','B3','B4','B5','C0','C1','C2','C3','C4','C5','D0','D1','D2','D3','D4','D5','E0','E1','E2','E3','E4','E5']]) 
df['level_ord'] = encoder.fit_transform(df[['level']].values)

df

/d/GH/GitWorkSpace/bank_model_competiton/data/v11
    title  career  zip_code  residence   loan  term  interest_rate  \
id                                                                   
0       9     0.0    221373          1   7200    36          10.95   
1       8    10.0    311681          0  21300    36          12.95   
2       8     7.0    271562          1  10400    60          21.05   
3       7     2.0    522083          0  33050    36          16.40   
4       8     3.0    101026          1   5200    36          14.35   

    issue_time  syndicated  installment  record_time  history_time  \
id                                                                   
0   1238631967           0            1   1238630622     472006661   
1   1128212052           0            0   1161907665     763779041   
2   1249171509           0            0   1383958593     727143443   
3   1172882234           0            1   1214353935     687660346   
4   1172882384           0            0

,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,...,loan_log,balance_account_avg_log,loan_term_avg_log,balance_accounts_ratio_log,interest_rate_log,balance_log,balance_limit_log,balance_accounts_log,level_hash,level_ord
id,,,,,,,,,,,,,,,,,,,,,
0,9,0.0,221373,1,7200,36,10.95,1238631967,0,1,...,8.881836,7.339249,5.298317,-0.635989,2.393339,9.536474,10.496814,2.197225,668,4.0
1,8,10.0,311681,0,21300,36,12.95,1128212052,0,0,...,9.966462,7.333241,6.382943,-0.635989,2.561096,9.530465,9.923290,2.197225,778,6.0
2,8,7.0,271562,1,10400,60,21.05,1249171509,0,0,...,9.249561,5.415112,5.155217,-0.635989,3.046901,7.612337,9.287301,2.197225,63,10.0
3,7,2.0,522083,0,33050,36,16.40,1172882234,0,1,...,10.405777,7.801209,6.822258,-0.635989,2.797281,9.998434,10.114559,2.197225,825,9.0
4,8,3.0,101026,1,5200,36,14.35,1172882384,0,0,...,8.556414,5.222755,4.972895,-0.635989,2.663750,7.419980,8.536996,2.197225,591,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53475,2,2.0,603000,1,9000,12,23.55,1172880000,0,0,...,9.104980,6.252185,6.620073,-0.875469,3.159126,7.861623,8.170469,1.609438,668,4.0
53476,0,10.0,601702,1,8000,12,30.70,1160092800,0,0,...,8.987197,6.574615,6.502290,-0.916291,3.424263,7.267763,7.583248,0.693147,591,8.0
53477,2,10.0,602808,1,10000,12,9.40,1180310400,0,0,...,9.210340,6.636941,6.725434,-0.875469,2.240710,8.246379,8.889170,1.609438,591,8.0


#  交易特征处理

In [38]:
df_tx = pd.read_csv(tx_input_path, nrows = NROWS, index_col=['id'])
df_tx['time_format'] = df_tx['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))

print(df['record_time'])
print(df_tx)

# 假设 ：issue time 是记录违约的时间，那只能用在此之前的流水
df_tx_merge = pd.merge(df_tx, df[['record_time','issue_time']], on = 'id', how='left')
df_tx = df_tx_merge[ df_tx_merge['time'] >  df_tx_merge['issue_time'] ]

id
0        1238630622
1        1161907665
2        1383958593
3        1214353935
4        1240274527
            ...    
53475    1157587200
53476    1138665600
53477    1108771200
53478    1159660800
53479    1159401600
Name: record_time, Length: 53480, dtype: int64
             time  direction       amount         time_format
id                                                           
0      1224115200          0  8771.350000 2008-10-16 08:00:00
0      1224288000          1   310.650000 2008-10-18 08:00:00
0      1224460800          1   152.620000 2008-10-20 08:00:00
0      1225152000          1    20.490000 2008-10-28 08:00:00
0      1226793600          1   173.170000 2008-11-16 08:00:00
...           ...        ...          ...                 ...
50371  1159747200          1   133.818723 2006-10-02 08:00:00
50371  1160179200          1    71.777279 2006-10-07 08:00:00
50371  1160956800          0    46.981217 2006-10-16 08:00:00
50371  1161129600          1   160.510592 2006-1

In [39]:
#days_diff, tm_count, total_amount, amount_1, amount_0, total_amount_day_avg, amount_1_day_avg, amount_0_day_avg
print(df_tx.shape)

df_tx_stat = pd.DataFrame()
df_tx_stat['tx_tmstp_max'] = df_tx.groupby('id')['time'].agg('max')
df_tx_stat['tx_tmstp_min'] = df_tx.groupby('id')['time'].agg('min')

df_tx_stat['tx_time_max'] = df_tx.groupby('id')['time_format'].agg('max')
df_tx_stat['tx_time_min'] = df_tx.groupby('id')['time_format'].agg('min')

df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat['tx_count'] = df_tx.groupby('id')['amount'].agg('count')
df_tx_stat['total_amount'] = df_tx.groupby('id')['amount'].agg('sum')

df_tx_stat['1_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 1]['amount'].sum())
df_tx_stat['0_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 0]['amount'].sum())

#交易活跃度： 效果次数/天数
# df_tx_stat['tx_count_avg']  = df_tx_stat['tx_count'] / df_tx_stat['tx_max_min_days'] 

df_tx_stat['total_amount_avg']  = df_tx_stat['total_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['1_amount_avg']  = df_tx_stat['1_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['0_amount_avg']  = df_tx_stat['0_amount'] / df_tx_stat['tx_max_min_days'] 

df_tx_stat['total_amount_avg2']  = df_tx_stat['total_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['1_amount_avg2']  = df_tx_stat['1_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['0_amount_avg2']  = df_tx_stat['0_amount'] / df_tx_stat['tx_count'] 


#log 处理
df['interest_rate_log'] = df['interest_rate'].apply(lambda x : np.log(x))


df_tx_stat.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)
df_tx_stat

(843348, 6)


,tx_tmstp_max,tx_tmstp_min,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2
id,,,,,,,,,,,,,
2,1383955200,1368403200,180,48,22406.100000,15883.720000,6522.380000,124.478333,88.242889,36.235444,466.793750,330.910833,135.882917
4,1238284800,1223683200,169,93,51163.000000,30823.100000,20339.900000,302.739645,182.385207,120.354438,550.139785,331.431183,218.708602
7,1201996800,1186876800,175,66,59958.010000,22642.760000,37315.250000,342.617200,129.387200,213.230000,908.454697,343.072121,565.382576
11,1170374400,1155686400,170,83,47281.180000,26862.430000,20418.750000,278.124588,158.014294,120.110294,569.652771,323.643735,246.009036
12,1209600000,1195257600,166,32,22975.850000,14362.920000,8612.930000,138.408735,86.523614,51.885120,717.995313,448.841250,269.154063
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53442,1161475200,1145923200,180,175,69716.586080,49026.153648,20690.432433,387.314367,272.367520,114.946847,398.380492,280.149449,118.231042
53451,1161475200,1146873600,169,29,6396.011204,6396.011204,0.000000,37.846220,37.846220,0.000000,220.552110,220.552110,0.000000
53465,1161475200,1153008000,98,7,1415.955972,1415.955972,0.000000,14.448530,14.448530,0.000000,202.279425,202.279425,0.000000


In [40]:
df_tx_stat.shape

(16130, 13)

# 合并交易特征

In [41]:
df_concat = pd.concat([df, df_tx_stat], axis=1)
print(df_concat.shape)
df_concat.reset_index(inplace=True)

(53480, 44)


In [42]:
df_concat = df_concat.rename(columns={'index':'id'})
print(df_concat)
df_concat.to_csv(output_path, index=False)
print('output_path : ', output_path)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
!ls

          id  title  career  zip_code  residence   loan  term  interest_rate  \
0          0      9     0.0    221373          1   7200    36          10.95   
1          1      8    10.0    311681          0  21300    36          12.95   
2          2      8     7.0    271562          1  10400    60          21.05   
3          3      7     2.0    522083          0  33050    36          16.40   
4          4      8     3.0    101026          1   5200    36          14.35   
...      ...    ...     ...       ...        ...    ...   ...            ...   
53475  53475      2     2.0    603000          1   9000    12          23.55   
53476  53476      0    10.0    601702          1   8000    12          30.70   
53477  53477      2    10.0    602808          1  10000    12           9.40   
53478  53478      0    10.0    602102          2   9000    12          24.40   
53479  53479      0    10.0    602207          1  10000    12          17.60   

       issue_time  syndicated  ...  tx_

In [43]:
!wc -l $output_path

53481 train.dat.v11


In [44]:
print('done  time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
print(df_concat.columns, '\n', df_concat.shape)
df_concat

done  time :  2025-08-28 14:49:37
Index(['id', 'title', 'career', 'zip_code', 'residence', 'loan', 'term',
       'interest_rate', 'issue_time', 'syndicated', 'installment',
       'record_time', 'history_time', 'total_accounts', 'balance_accounts',
       'balance_limit', 'balance', 'level', 'label', 'balance_account_avg',
       'loan_term_avg', 'balance_accounts_ratio', 'loan_log',
       'balance_account_avg_log', 'loan_term_avg_log',
       'balance_accounts_ratio_log', 'interest_rate_log', 'balance_log',
       'balance_limit_log', 'balance_accounts_log', 'level_hash', 'level_ord',
       'tx_tmstp_max', 'tx_tmstp_min', 'tx_max_min_days', 'tx_count',
       'total_amount', '1_amount', '0_amount', 'total_amount_avg',
       '1_amount_avg', '0_amount_avg', 'total_amount_avg2', '1_amount_avg2',
       '0_amount_avg2'],
      dtype='object') 
 (53480, 45)


,id,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,...,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2
0,0,9,0.0,221373,1,7200,36,10.95,1238631967,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,8,10.0,311681,0,21300,36,12.95,1128212052,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8,7.0,271562,1,10400,60,21.05,1249171509,0,...,48.0,22406.1,15883.72,6522.38,124.478333,88.242889,36.235444,466.793750,330.910833,135.882917
3,3,7,2.0,522083,0,33050,36,16.40,1172882234,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,8,3.0,101026,1,5200,36,14.35,1172882384,0,...,93.0,51163.0,30823.10,20339.90,302.739645,182.385207,120.354438,550.139785,331.431183,218.708602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53475,53475,2,2.0,603000,1,9000,12,23.55,1172880000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53476,53476,0,10.0,601702,1,8000,12,30.70,1160092800,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53477,53477,2,10.0,602808,1,10000,12,9.40,1180310400,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53478,53478,0,10.0,602102,2,9000,12,24.40,1176768000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
